## Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import normalize

from nltk.tokenize import WordPunctTokenizer

from tqdm import tqdm

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title;
 - description;
 - Category_name;
 - Category.

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [4]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [5]:
data.shape

(30000, 4)

In [6]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

## Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [2]:
tokenizer = WordPunctTokenizer()


def preprocess_wpt(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess_wpt(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [14]:
def tokenize_X_wpt(X):
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            X[i][j] = preprocess_wpt(X[i][j])
    return X

In [15]:
X_train_wpt = tokenize_X_wpt(X_train)
X_test_wpt = tokenize_X_wpt(X_test)

In [16]:
X_train_wpt[:11]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [17]:
assert X_train_wpt[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите 10000 самых частых слов из title и description обучающей выборки, отсортируйте их по убыванию частотности.

In [18]:
def frequency_words(X):
    counter_words = {}
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            for word in X[i][j].split(' '):
                word = word.strip(' ')
                if word in counter_words:
                    counter_words[word] += 1
                else:
                    counter_words[word] = 1
    return counter_words

In [19]:
bow_vocabulary = frequency_words(X_train_wpt)
bow_vocabulary

{'сапоги': 454,
 '46': 471,
 'размер': 3320,
 'новые': 1872,
 'светильники': 26,
 'потолочный': 8,
 'swarovski': 9,
 'потолочные': 8,
 '6': 2126,
 'штук': 135,
 ',': 79117,
 'цена': 2379,
 'за': 3425,
 'штуку': 134,
 '.': 65624,
 'в': 28337,
 'эксплуатации': 157,
 '2': 5766,
 'года': 1012,
 'продаются': 152,
 'связи': 322,
 'со': 853,
 'сменой': 10,
 'интерьера': 39,
 'квартире': 261,
 'iphone': 465,
 '7': 1485,
 'plus': 163,
 '128gb': 25,
 'red': 34,
 'красный': 121,
 'наличии': 2172,
 '/': 85802,
 'данная': 149,
 'только': 1129,
 'для': 9627,
 'подписчиков': 2,
 'instagram': 39,
 ':': 15098,
 'iqmac': 1,
 'новый': 1626,
 'айфон': 37,
 'это': 744,
 'элегантный': 14,
 'и': 21714,
 'мощный': 53,
 'смартфон': 70,
 'который': 200,
 'готов': 82,
 'полной': 71,
 'мере': 9,
 'раскрыть': 4,
 'возможности': 70,
 'ios': 37,
 '10': 2452,
 'аппарат': 117,
 'с': 12860,
 '4': 2939,
 '-': 36840,
 'ядерным': 1,
 'процессором': 5,
 'а10': 2,
 '3': 4071,
 'гб': 213,
 'озу': 24,
 'легкостью': 13,
 'реша

In [20]:
bow_vocabulary = sorted(bow_vocabulary, key=bow_vocabulary.get, reverse=True)[:10000] # конструкция взята со stackoverflow

In [21]:
bow_vocabulary[:5]

['/', ',', '.', '-', 'в']

In [22]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [23]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из bow_vocabulary
    указано количество его употреблений
    """ 
    zero_vector = {}
    for i in bow_vocabulary:
        zero_vector[i] = 0
    
    for word in text.split(' '):
        word = word.strip(' ')
        if word in zero_vector:
            zero_vector[word] += 1
    
    return np.array(list(zero_vector.values()))

In [24]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [25]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    
    items_bow = []
    for item in range(len(items)):
        item_bow = text_to_bow(items[item][1])
        items_bow.append(item_bow)
    return np.array(items_bow)

In [26]:
assert np.allclose(np.where(items_to_bow([X_train_wpt[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),
)

In [27]:
X_train_bow_wpt = items_to_bow(X_train_wpt)
X_test_bow_wpt = items_to_bow(X_test_wpt)

In [28]:
X_train_bow_wpt[:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 2, 1, ..., 0, 0, 0],
       [4, 5, 3, ..., 0, 0, 0],
       [7, 4, 2, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


Подсказка: рассмотрите использование sparse-матриц - в них хранятся только ненулевые элементы.  
scipy.sparse.csr_matrix(X_train_bow)

In [30]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow_wpt, y_train)
print(accuracy_score(bow_model.predict(X_test_bow_wpt), y_test))

assert accuracy_score(bow_model.predict(X_test_bow_wpt), y_test) > 0.7

C:\Users\kiril\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\kiril\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7046666666666667


In [31]:
bow_model = LinearSVC(max_iter=70).fit(X_train_bow_wpt, y_train)
print(accuracy_score(bow_model.predict(X_test_bow_wpt), y_test))

assert accuracy_score(bow_model.predict(X_test_bow_wpt), y_test) > 0.68

C:\Users\kiril\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6844444444444444


### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?  
Постройте признаковое пространство на конкатенации (через пробел) title и description.

In [32]:
def items_to_bow_full(items: np.array) -> np.array:
    items_bow = []
    for item in range(items.shape[0]):
        item_bow_title = text_to_bow(items[item][0])
        item_bow_desc = text_to_bow(items[item][1])
        item_bow = item_bow_title + item_bow_desc
        items_bow.append(item_bow)
    return np.array(items_bow)

In [33]:
X_train_bow_wpt = items_to_bow_full(X_train_wpt)
X_test_bow_wpt = items_to_bow_full(X_test_wpt)

### Logistic Regression (после добавления title)

In [35]:
bow_model = LogisticRegression().fit(X_train_bow_wpt, y_train)
print(accuracy_score(bow_model.predict(X_test_bow_wpt), y_test))

C:\Users\kiril\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\kiril\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7837777777777778


### SVC (после добавления title)

In [36]:
bow_model = LinearSVC().fit(X_train_bow_wpt, y_train)
print(accuracy_score(bow_model.predict(X_test_bow_wpt), y_test))

C:\Users\kiril\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7533333333333333


__Вывод:__ качество увеличилось, так как после добавление слов из другого столбца наша модель стала лучше классифицировать товары, придав больше важности каким-то словам, которое были одновременно в заголовке и тексте.

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством?

In [38]:
X_train_norm = normalize(X_train_bow_wpt)
X_test_norm = normalize(X_test_bow_wpt)

### Logistic Regression (+после нормализации)

In [39]:
bow_model = LogisticRegression().fit(X_train_norm, y_train)
print(accuracy_score(bow_model.predict(X_test_norm), y_test))

C:\Users\kiril\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\kiril\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6622222222222223


### SVC (+после нормализации)

In [40]:
bow_model = LinearSVC().fit(X_train_norm, y_train)
print(accuracy_score(bow_model.predict(X_test_norm), y_test))

0.7987777777777778


__Вывод__: после нормализации качество на SVC подросло с 0.75 до 0.799, а в случае с LR - упало с 0.78 до 0.66.